In [13]:
import numpy as np
import math
from kaggle_environments.envs.halite.helpers import *
from kaggle_environments import make

In [11]:
def world_feature(board):
    size = board.configuration.size
    me = board.current_player
    
    ships = np.zeros((1, size, size))
    ship_cargo = np.zeros((1, size, size))
    bases = np.zeros((1, size, size))

    map_halite = np.array(board.observation['halite']).reshape(1, size, size)/1000

    for iid, ship in board.ships.items():
        ships[0, ship.position[1], ship.position[0]] = 1 if ship.player_id == me.id else -1
        ship_cargo[0, ship.position[1], ship.position[0]] = ship.halite/1000

    for iid, yard in board.shipyards.items():
        bases[0, yard.position[1], yard.position[0]] = 1 if yard.player_id == me.id else -1
        
    return np.concatenate([
        map_halite, 
        ships, 
        ship_cargo, 
        bases
    ], axis=0)

In [22]:
NUM_AGENTS = 2
BOARD_SIZE = 7
TURNS = 400

env = make('halite', configuration={"randomSeed": 1, "episodeSteps": TURNS, "size": BOARD_SIZE}, debug=True)

In [122]:
sample_obs = env.state[0].observation
board = Board(sample_obs, env.configuration)
features = world_feature(board)
cargo = features[2]
cargo[0][1]

0.0

In [6]:
def getDirTo(fromPos, toPos, size):
    fromX, fromY = divmod(fromPos[0],size), divmod(fromPos[1],size)
    toX, toY = divmod(toPos[0],size), divmod(toPos[1],size)
    if fromY < toY: return ShipAction.NORTH
    if fromY > toY: return ShipAction.SOUTH
    if fromX < toX: return ShipAction.EAST
    if fromX > toX: return ShipAction.WEST

In [7]:
# Find nearest objects to ships
def nearestObject(fromPos, positions, size):
    fromX, fromY = fromPos
    distance = []
    for obj, halite in positions:
        toX, toY = obj
        distance.append(math.sqrt((toX-fromX)**2 + (toY-fromY)**2))
    nearest = min(distance)
    index = distance.index(nearest)
    return positions[index]

In [9]:
NUM_AGENTS = 2
BOARD_SIZE = 7
TURNS = 400

env = make('halite', configuration={"randomSeed": 1, "episodeSteps": TURNS, "size": BOARD_SIZE}, debug=True)
board = Board(obs, config)

NameError: name 'obs' is not defined

In [ ]:
ship_states[ship.id] -> geht find_best_halite

In [69]:
def manhattanDist(X1, Y1, X2, Y2):
    dist = math.fabs(X2 - X1) + math.fabs(Y2 - Y1)
    return (int)(dist)

In [70]:
def get_halite_pos(halite_map):
    list_of_pos = []
    for i in range(halite_map.shape[0]):
        for j in range(halite_map.shape[1]):
            element = halite_map[i][j]
            if element > 0:
                pos_tuple = (i,j, element)
                list_of_pos.append(pos_tuple)
    return list_of_pos


In [71]:

halie = get_halite_pos(features[0])
type(halie)

list

In [140]:
def find_halite(ship, halite_map):
    distance = []
    score = []
    halite_score = []
    me = board.current_player
    for index, tuple in enumerate(halite_map):
        score = tuple[2]
        distance = manhattanDist(X1=ship.position[0],Y1=ship.position[1],X2= tuple[0], Y2= tuple[1])
        result_tuple = (distance,score, tuple[0],tuple[1])
        halite_score.append(result_tuple)
    return halite_score

In [104]:
halite = find_halite()
halite

([(5, 1.825, 1, 0),
  (4, 2.281, 1, 1),
  (2, 0.837, 1, 3),
  (4, 2.281, 1, 5),
  (5, 1.825, 1, 6),
  (4, 0.664, 2, 0),
  (3, 0.247, 2, 1),
  (1, 0.058, 2, 3),
  (3, 0.247, 2, 5),
  (4, 0.664, 2, 6),
  (1, 0.026, 3, 2),
  (0, 0.058, 3, 3),
  (1, 0.026, 3, 4),
  (4, 0.664, 4, 0),
  (3, 0.247, 4, 1),
  (1, 0.058, 4, 3),
  (3, 0.247, 4, 5),
  (4, 0.664, 4, 6),
  (5, 1.825, 5, 0),
  (4, 2.281, 5, 1),
  (2, 0.837, 5, 3),
  (4, 2.281, 5, 5),
  (5, 1.825, 5, 6)],)

In [139]:
def calc_best_pos_halite(halite):
    result_list = []
    for index, tuple in enumerate(halite[0]):
        distance = tuple[0]
        value = tuple[1]
        result = distance * value
        new_result = result,tuple[2], tuple[3]
        result_list.append(new_result)
    min_value = optimize(result_list)
    print(min_value)
    for index, tuple in enumerate(halite[0]):
        if tuple[0] == min_value:
            pos = tuple[2], tuple[3]
    return pos

In [137]:
def optimize(value_list):
    tmp_list = []
    for elements in range(len(value_list)):
        tmp_list.append(result_list[elements][0])
    min_value = min(tmp_list)
    return min_value

In [138]:
calc_pos_halite(halite)

0.0


(3, 3)

In [ ]:

def mining():
    """find coordination of best halite storage"""
    size = config.size
    me = board.current_player
    ship_states = {}
    for ship in me.ships:
        pos_halite = get_halite_pos(halite_map)
        list_of_halite_pos_and_distance = find_halite(ship,pos_halite)
        best_halite = calc_best_pos_halite(list_of_halite_pos_and_distance)
        direction = getDirTo(ship.position, best_halite, size)
        if direction:
            ship.next_action = direction
